# ライブラリのインポート


In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from statsbombpy import sb 
from mplsoccer import Pitch 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pycaret
from pycaret.classification import *
from imblearn.over_sampling import SMOTENC
import mlflow
import dagshub

# データセットの準備と欠損値処理

In [2]:
#データの読み込み(statsbomb/open-dataから)

event = sb.events(match_id=3893827) #試合を新しく取得する場合は"sb.competition()->sb.matches(competion_id=,season_id=)

#print(event)

In [3]:
#eventをpycaretに認識しやすい形　+　SMOTEで不均衡データのオーバーサンプリングを実施できるようにする
#DataFrameの複製
df2 = event.copy()

#データの抽出(ball_receipt_outcome)

df2 = df2[df2['type'] == 'Ball Receipt*']

df2['ball_receipt_outcome'] = df2['ball_receipt_outcome'].fillna('Complete')


# 結果の確認
#print(df2)

event['ball_receipt_outcome'] = df2['ball_receipt_outcome']

event['ball_receipt_outcome'] = event.ball_receipt_outcome.fillna('0').astype('category')
event['ball_receipt_outcome'] = event.ball_receipt_outcome.fillna('0').astype('category')
event.ball_recovery_recovery_failure = event.ball_recovery_recovery_failure.fillna(False)
event['block_deflection'] = event['block_deflection'].fillna(False)
event['clearance_aerial_won'] = event['clearance_aerial_won'].fillna(False)#.astype('boolean')
event['clearance_head'] = event['clearance_head'].fillna(False)#.astype('boolean')
event['clearance_right_foot'] = event['clearance_right_foot'].fillna(False)#.astype('boolean')
event['clearance_left_foot'] = event['clearance_left_foot'].fillna(False)#.astype('boolean')
event['clearance_body_part'] = event['clearance_body_part'].fillna('0').astype('category')
event['dribble_overrun'] = event['dribble_overrun'].fillna(False)
event['foul_committed_advantage'] = event['foul_committed_advantage'].fillna(False)#.astype('boolean')
event['foul_committed_card'] = event['foul_committed_card'].fillna('No Card')
event['foul_committed_card'] = event['foul_committed_card'].fillna('0').astype('category') 
event['foul_committed_offensive'] = event['foul_committed_offensive'].fillna(False)
event['foul_won_advantage'] = event['foul_won_advantage'].fillna(False)#.astype('boolean')
event['foul_won_defensive'] = event['foul_won_defensive'].fillna(False)
#goalkeepr_end_potition
df2 = event[event['type'] == 'Goal Keeper' ]

event_X_Y= df2['goalkeeper_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
df2['goalkeeper_end_location_x'] = event_X_Ydf[0]

#y座標の代入
df2['goalkeeper_end_location_y'] = event_X_Ydf[1]

#元のdataに結合

event['goalkeeper_end_location_x'] = df2['goalkeeper_end_location_x']
event['goalkeeper_end_location_y'] = df2['goalkeeper_end_location_y']

#carry_end_location
df2 = event[event['type'] == 'Carry']

event_X_Y= df2['carry_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
df2['carry_end_location_x'] = event_X_Ydf[0]

#y座標の代入
df2['carry_end_location_y'] = event_X_Ydf[1]

#元のdataに結合

event['carry_end_location_x'] = df2['carry_end_location_x']
event['carry_end_location_y'] = df2['carry_end_location_y']

#location
event_X_Y= event['location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
event['location_x'] = event_X_Ydf[0]

#y座標の代入
event['location_y'] = event_X_Ydf[1]

event['off_camera'] = event['off_camera'].fillna(False)
event['out'] = event['out'].fillna(False)
event['counterpress'] = event['counterpress'].fillna(False)
event['pass_aerial_won'] = event['pass_aerial_won'].fillna(False)#.astype('boolean')
event['pass_cross'] = event['pass_cross'].fillna(False)#.astype('boolean')
event['pass_cut_back'] = event['pass_cut_back'].fillna(False)#.astype('boolean')
event['pass_goal_assist'] = event['pass_goal_assist'].fillna(False)#.astype('boolean')
event['pass_inswinging'] = event['pass_inswinging'].fillna(False)#.astype('boolean')
event['pass_no_touch'] = event['pass_no_touch'].fillna(False)#.astype('boolean')
event['pass_outswinging'] = event['pass_outswinging'].fillna(False)#.astype('boolean')
event['pass_through_ball'] = event['pass_through_ball'].fillna(False)#.astype('boolean')
event['pass_shot_assist'] = event['pass_shot_assist'].fillna(False)#.astype('boolean')
event['pass_straight'] = event['pass_straight'].fillna(False)#.astype('boolean')
event['pass_switch'] = event['pass_switch'].fillna(False)


event_X_Y= event['pass_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
event['pass_end_location_x'] = event_X_Ydf[0]

#y座標の代入
event['pass_end_location_y'] = event_X_Ydf[1]


event['shot_aerial_won'] = event['shot_aerial_won'].fillna(False)#.astype('boolean')
event['shot_first_time'] = event['shot_first_time'].fillna(False)#.astype('boolean')
event['shot_one_on_one'] = event['shot_one_on_one'].fillna(False)
event['under_pressure'] = event['under_pressure'].fillna(False)

#shot_end_location
event_X_Y= event['shot_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#x座標を代入
event['shot_end_location_x'] = event_X_Ydf[0]

#y座標の代入
event['shot_end_location_y'] = event_X_Ydf[1]

#z座標の取得
event['shot_end_location_z'] = event_X_Ydf[2]

#元のdataに結合

#event['goalkeeper_end_location_x'] = df2['goalkeeper_end_location_x']
#event['goalkeeper_end_location_y'] = df2['goalkeeper_end_location_y']

event.dribble_outcome = event.dribble_outcome.fillna('0').astype('category')
event.duel_outcome = event.duel_outcome.fillna('0').astype('category')
event.duel_type =event.duel_type.fillna('0').astype('category')
event.duration = event.duration.fillna('0')#.astype('float')
event.foul_committed_type = event.foul_committed_type.fillna('0').astype('category')
event.goalkeeper_body_part = event.goalkeeper_body_part.fillna('0').astype('category')
event.goalkeeper_outcome = event.goalkeeper_outcome.fillna('0').astype('category')
event.goalkeeper_position = event.goalkeeper_position.fillna('0').astype('category')
event.goalkeeper_technique = event.goalkeeper_technique.fillna('0').astype('category')
event.goalkeeper_type = event.goalkeeper_type.fillna('0').astype('category')
event.interception_outcome = event.interception_outcome.fillna('0').astype('category')
event.pass_assisted_shot_id = event.pass_assisted_shot_id.fillna('0').astype('category')
event.pass_body_part = event.pass_body_part.fillna('0').astype('category')
event.pass_height = event.pass_height.fillna('0').astype('category')
event.pass_length = event.pass_length.fillna('0').astype('category')
event.pass_outcome = event.pass_outcome.fillna('0').astype('category')
event.pass_recipient = event.pass_recipient.fillna('0').astype('category')
event.pass_recipient_id = event.pass_recipient_id.fillna('0').astype('category')
event.pass_technique = event.pass_technique.fillna('0').astype('category')
event.pass_type = event.pass_type.fillna('0').astype('category')
event.player = event.player.fillna('No player').astype('category')
event.player_id = event.player_id.fillna('0').astype('int')
event.position = event.position.fillna('0').astype('category')
event.shot_body_part = event.shot_body_part.fillna('0').astype('category')
event.shot_key_pass_id = event.shot_key_pass_id.fillna('0').astype('category')
event.shot_outcome = event.shot_outcome.fillna('0').astype('category')
event.shot_statsbomb_xg = event.shot_statsbomb_xg.fillna('0.0').astype('float')
event.shot_technique = event.shot_technique.fillna('0').astype('category')
event.substitution_replacement = event.substitution_replacement.fillna('0').astype('category')
event.substitution_replacement_id = event.substitution_replacement_id.fillna('0').astype('int')
event.play_pattern = event.play_pattern.astype('category')
event.id = event.id.astype('category')
event.possession_team = event.possession_team.astype('category')
event.team = event.team.astype('category')


event = event.drop(['shot_type', 'substitution_outcome_id','substitution_outcome',
                    'related_events', 'tactics', 'shot_freeze_frame','shot_end_location',
                    'pass_end_location','injury_stoppage_in_chain',
                    'goalkeeper_end_location', 'carry_end_location','location'],
                    axis=1)


#event

# 目的変数の設定と変換

In [4]:
event_fi = event.drop(['pass_angle', 'carry_end_location_x', 'carry_end_location_y', 'goalkeeper_end_location_x', 
                       'goalkeeper_end_location_y', 'pass_end_location_x', 'pass_end_location_y', 
                       'shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z'], axis=1)

event_fi = event_fi.replace({'type':{'Carry':'Dribble'}})

#replace({'age': [24, 18], 'point': 70}, 100))

event_fi = event_fi.replace({'type':['Ball Receipt*', 'Pressure','Ball Recovery', 'Duel', 'Goal Keeper', 'Shot', 'Block', 'Miscontrol',  'Clearance', 'Dispossessed', 'Interception', 'Foul Committed', 'Dribbled Past', 'Foul Won', 'Substitution','Half Start', 'Half End', 'Injury Stoppage', 'Player Off','Player On','Referee Ball-Drop','Starting XI', 'Tactical Shift', 'Shield', 'Error' ]},'Others')

#event_fi
#event_fi.info()

le = LabelEncoder()
le.fit(event_fi['type'])

event_fi['type_num'] = le.transform(event_fi['type'])

event_fi

,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,dribble_outcome,...,substitution_replacement,substitution_replacement_id,team,team_id,timestamp,type,under_pressure,location_x,location_y,type_num
0,0,False,False,False,0,False,False,False,False,0,...,0,0,Vietnam Women's,16769,00:00:00.000,Others,False,NaN,NaN,1
1,0,False,False,False,0,False,False,False,False,0,...,0,0,Netherlands Women's,851,00:00:00.000,Others,False,NaN,NaN,1
2,0,False,False,False,0,False,False,False,False,0,...,0,0,Vietnam Women's,16769,00:00:00.000,Others,False,NaN,NaN,1
3,0,False,False,False,0,False,False,False,False,0,...,0,0,Netherlands Women's,851,00:00:00.000,Others,False,NaN,NaN,1
4,0,False,False,False,0,False,False,False,False,0,...,0,0,Vietnam Women's,16769,00:00:00.000,Others,False,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3972,0,False,False,False,0,False,False,False,False,0,...,0,0,Netherlands Women's,851,00:05:02.341,Others,False,32.8,64.2,1
3973,0,False,False,False,0,False,False,False,False,0,...,0,0,Vietnam Women's,16769,00:05:02.518,Others,False,87.3,15.9,1
3974,0,False,False,False,0,False,False,False,False,0,...,0,0,Vietnam Women's,16769,00:05:25.397,Others,False,NaN,NaN,1
3975,0,False,False,False,0,False,False,False,False,0,...,0,0,Vietnam Women's,16769,00:18:00.681,Others,False,NaN,NaN,1


# pycaret用オブジェクトとデータ分割、学習

In [5]:

event_fi_py = event_fi.drop('timestamp', axis=1)

In [6]:
ml_data = event_fi_py.sample(frac=0.8, random_state=5).reset_index(drop=True)
ml_data_unseen = event_fi_py.drop(ml_data.index).reset_index(drop=True)


In [ ]:
#実行はしない
ml_data_unseen

In [7]:
exp_trial = setup(data = ml_data, target= 'type_num' ,
                  ignore_features = ["type","id", "pass_recipient","pass_recipient_id", "pass_aerial_won", "pass_assisted_shot_id",
                                     "pass_body_part", "pass_height", "pass_inswinging", "pass_length", "pass_type", "player_id", "team_id"], 
                  fix_imbalance = True,
                  log_experiment = "dagshub", experiment_name = 'trial_classification_2', log_plots = True, log_data =True,
                  session_id= 1)

,Description,Value
0,Session id,1
1,Target,type_num
2,Target type,Multiclass
3,Original data shape,"(3182, 78)"
4,Transformed data shape,"(4270, 146)"
5,Transformed train set shape,"(3315, 146)"
6,Transformed test set shape,"(955, 146)"
7,Ignore features,13
8,Numeric features,11
9,Categorical features,26


Accessing as TT9625

Initialized MLflow to track repo "TT9625/aa"

Repository TT9625/aa initialized!

2024/11/05 14:36:12 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Uploading files (2) to "TT9625/aa"...

Upload successful, content was identical and no new commit was created

## compare_models

In [8]:
compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:58:36
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.6866,0.0000,0.6866,0.6745,0.6480,0.5090,0.5505,0.4750
qda,Quadratic Discriminant Analysis,0.6529,0.0000,0.6529,0.8517,0.6529,0.4828,0.5562,0.4660
et,Extra Trees Classifier,0.5721,0.8817,0.5721,0.6058,0.4922,0.2001,0.2791,0.7140
rf,Random Forest Classifier,0.5680,0.8853,0.5680,0.7464,0.4681,0.1710,0.3003,0.7630
dt,Decision Tree Classifier,0.5110,0.5147,0.5110,0.5347,0.3598,0.0367,0.1313,0.4350
ada,Ada Boost Classifier,0.5110,0.0000,0.5110,0.5347,0.3598,0.0367,0.1313,0.6990
gbc,Gradient Boosting Classifier,0.5110,0.0000,0.5110,0.5347,0.3598,0.0367,0.1313,1.9880
xgboost,Extreme Gradient Boosting,0.5110,0.6247,0.5110,0.5347,0.3598,0.0367,0.1313,0.8030
lightgbm,Light Gradient Boosting Machine,0.5110,0.7056,0.5110,0.5347,0.3598,0.0367,0.1313,1.4310
lda,Linear Discriminant Analysis,0.4526,0.0000,0.4526,0.3317,0.3279,-0.0461,-0.0838,0.4850


2024/10/30 18:01:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 18:01:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run Ridge Classifier at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/2/runs/33abe88a4e7f4b4faf8944da521a0d31.
2024/10/30 18:01:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/2.
2024/10/30 18:01:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 18:02:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run Quadratic Discriminant Analysis at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/2/runs/30fd4349ceb041af8df4a529698b068a.
2024/10/30 18:02:45 INFO mlflow.tracking._tracking_service.client: 🧪

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=1, solver='auto',
                tol=0.0001)

## extra Tree Classfier (et)

In [9]:
et = create_model(et)


KeyboardInterrupt



In [19]:
tune_et = tune_model(et,optimize='AUC')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:02:12
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7399,0.9089,0.7399,0.8271,0.7227,0.5912,0.6277
1,0.7309,0.9110,0.7309,0.7309,0.7272,0.5741,0.5773
2,0.7309,0.9180,0.7309,0.7187,0.7238,0.5650,0.5660
3,0.6682,0.8662,0.6682,0.6722,0.6655,0.4730,0.4773
4,0.6682,0.9142,0.6682,0.8003,0.6524,0.4810,0.5214
5,0.7309,0.8970,0.7309,0.7335,0.7282,0.5732,0.5767
6,0.7489,0.9017,0.7489,0.7748,0.7551,0.6079,0.6144
7,0.6712,0.9073,0.6712,0.7186,0.6342,0.4788,0.5134
8,0.7117,0.8929,0.7117,0.7203,0.7116,0.5400,0.5430


Fitting 10 folds for each of 10 candidates, totalling 100 fits


2024/10/28 10:02:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/28 10:03:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run Extra Trees Classifier at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/1/runs/82e990c815de43d8a764674f5a9a6a67.
2024/10/28 10:03:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/1.


In [20]:
evaluate_model(tune_et)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
finalize_model(tune_et)

In [24]:
test_classification_et = predict_model(tune_et, data=ml_data_unseen)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.7182,0,0,0,0,0.1586,0.2931


In [ ]:
test_classification_et

## ridge

In [27]:
ridge = create_model('ridge')

2024/10/28 23:58:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/28 23:58:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run Ridge Classifier at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/1/runs/6b98c3737bb24eacb24c0071f77a2c66.
2024/10/28 23:58:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/1.


In [28]:
tuned_ridge = tune_model(ridge, optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7668,0.0000,0.7668,0.7865,0.7638,0.6348,0.6477
1,0.7265,0.0000,0.7265,0.7550,0.7298,0.5766,0.5868
2,0.7623,0.0000,0.7623,0.7734,0.7540,0.6244,0.6378
3,0.6861,0.0000,0.6861,0.7008,0.6842,0.5060,0.5149
4,0.7309,0.0000,0.7309,0.7630,0.7340,0.5809,0.5938
5,0.7489,0.0000,0.7489,0.7673,0.7472,0.6063,0.6169
6,0.7175,0.0000,0.7175,0.7547,0.7246,0.5630,0.5735
7,0.7613,0.0000,0.7613,0.7814,0.7643,0.6262,0.6331
8,0.7658,0.0000,0.7658,0.7900,0.7685,0.6350,0.6443


Fitting 10 folds for each of 10 candidates, totalling 100 fits


2024/10/29 00:10:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/29 00:10:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run Ridge Classifier at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/1/runs/ba77b815da6d42ac9fceef1f8ba01831.
2024/10/29 00:10:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/1.


In [29]:
evaluate_model(tuned_ridge)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [30]:
test_classification_ridge = predict_model(tuned_ridge, data= ml_data_unseen)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ridge Classifier,0.6403,0.8521,0,0,0,0.1217,0.2504


In [ ]:
test_classification_ridge

## rf

In [10]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5605,0.9074,0.5605,0.7666,0.4502,0.1492,0.2835
1,0.5695,0.8808,0.5695,0.7692,0.4643,0.1698,0.3044
2,0.5471,0.8767,0.5471,0.7629,0.4280,0.1180,0.2497
3,0.5740,0.8792,0.5740,0.7705,0.4780,0.1810,0.3149
4,0.5740,0.8849,0.5740,0.7705,0.4761,0.1807,0.3147
5,0.5605,0.8763,0.5605,0.7211,0.4630,0.1619,0.2832
6,0.5605,0.8697,0.5605,0.7676,0.4582,0.1585,0.2929
7,0.5856,0.9044,0.5856,0.7458,0.5055,0.2151,0.3354
8,0.5811,0.8900,0.5811,0.7730,0.4902,0.2009,0.3337


2024/10/30 20:04:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 20:04:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Classifier at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/2/runs/af67f641f9c14ff39ceb426509c2d6d1.
2024/10/30 20:04:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/2.


In [11]:
tuned_rf = tune_model(rf, optimize= 'F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7220,0.9136,0.7220,0.7978,0.6837,0.5463,0.5802
1,0.7175,0.9003,0.7175,0.8114,0.6898,0.5486,0.5843
2,0.6726,0.8695,0.6726,0.7664,0.6476,0.4709,0.4994
3,0.6143,0.9046,0.6143,0.6913,0.5507,0.2961,0.3694
4,0.6771,0.8702,0.6771,0.7881,0.6576,0.4881,0.5220
5,0.6861,0.9100,0.6861,0.7825,0.6537,0.4955,0.5288
6,0.6771,0.8909,0.6771,0.8182,0.6548,0.4990,0.5479
7,0.6937,0.9142,0.6937,0.7850,0.6620,0.5071,0.5398
8,0.7432,0.9208,0.7432,0.8194,0.7283,0.5929,0.6233


Fitting 10 folds for each of 10 candidates, totalling 100 fits


2024/10/30 20:06:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/30 20:06:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Classifier at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/2/runs/03ff6b1f12344691b71fdfffdce385c5.
2024/10/30 20:06:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/TT9625/aa.mlflow/#/experiments/2.


In [12]:
evaluate_model(tuned_rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [34]:
test_classification_rf = predict_model(tuned_rf, data=ml_data_unseen)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8264,0.9919,0.8194,1.0000,0.9007,0.2613,0.3877


In [ ]:
test_classification_rf